In [1]:
from pathlib import Path
import gzip
import pickle
import requests
import torch

In [2]:
url = "https://github.com/pytorch/tutorials/raw/master/_static/"
filename = "mnist.pkl.gz"

In [3]:
data_path = Path("data") if Path("data").exists() else Path("../data")
path = data_path / "downloaded"/"vector_mnist"
path.mkdir(parents=True,exist_ok=True)

In [4]:
if not (path/filename).exists():
    content = requests.get(url+filename).content
    (path/filename).open("wb").write(content) 

In [5]:
!cd ../ && tree

.
├── data
│   └── downloaded
│       ├── vector-mnist
│       │   └── mnist.pkl.gz
│       └── vector_mnist
│           └── mnist.pkl.gz
├── notebooks
│   ├── lab01_pytorch.ipynb
│   └── lab01_pytorch_self.ipynb
└── text_recognizer
    ├── __init__.py
    ├── __pycache__
    │   ├── __init__.cpython-310.pyc
    │   └── __init__.cpython-38.pyc
    ├── data
    │   └── util.py
    ├── metadata
    │   ├── __pycache__
    │   │   ├── mnist.cpython-310.pyc
    │   │   ├── mnist.cpython-38.pyc
    │   │   ├── shared.cpython-310.pyc
    │   │   └── shared.cpython-38.pyc
    │   ├── mnist.py
    │   └── shared.py
    ├── models
    │   ├── __init__.py
    │   └── mlp.py
    └── util.py

11 directories, 17 files


In [6]:
datafile = (path/filename)

In [7]:
with gzip.open(datafile,"rb") as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f,encoding="latin-1")


In [8]:
x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)

In [9]:
x_train[0,::2].shape

torch.Size([392])

In [10]:
x_train.ndim

2

In [11]:
# re-execute this cell for more samples
import sys
sys.path.append("/Users/convergeml/engineering/fsdl-text-recognizer-2022-labs/lab01")

import random
import wandb  # just for some convenience methods that convert tensors to human-friendly datatypes
import text_recognizer.metadata.mnist as metadata # metadata module holds metadata separate from data

In [12]:
idx = random.randint(0, len(x_train))
example = x_train[idx]

print(y_train[idx])  # the label of the image
wandb.Image(example.reshape(*metadata.DIMS)).image  # the image itself

tensor(2)


In [13]:
import math

In [14]:
weights = torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

In [15]:
def linear(x: torch.Tensor) -> torch.Tensor:
    return x @ weights + bias

In [16]:
def log_softmax(x: torch.Tensor) -> torch.Tensor:
    return x - torch.log(torch.sum(torch.exp(x), axis=1))[:, None]

def model(xb: torch.Tensor) -> torch.Tensor:
    return log_softmax(linear(xb))

In [17]:
#64*784 , 784 * 10
#64 * 10, 10
#64 * 10

In [18]:
bs = 64  # batch size

xb = x_train[0:bs]  # a batch of inputs
outs = model(xb)  # outputs on that batch

print(outs[0], outs.shape)  # outputs on the first element of the batch

tensor([-2.7285, -2.5916, -2.5317, -2.8019, -2.8102, -2.4919, -1.8417, -2.7460,
        -1.5848, -1.9045], grad_fn=<SelectBackward0>) torch.Size([64, 10])


In [19]:
def accuracy(out: torch.Tensor, yb: torch.Tensor) -> torch.Tensor:
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [20]:
yb = y_train[0:bs]

acc = accuracy(outs, yb)

print(acc)

tensor(0.1875)


In [21]:
try:
    acc.backward()
except RuntimeError as e:
    print(e)

element 0 of tensors does not require grad and does not have a grad_fn


In [22]:
def cross_entropy(output: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    return -output[range(target.shape[0]), target].mean()

loss_func = cross_entropy

In [23]:
print(type(outs),type(yb))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [24]:
print(
    loss_func(outs, yb), 
-torch.log(torch.tensor(1 / 10))
)

tensor(2.3211, grad_fn=<NegBackward0>) tensor(2.3026)


In [25]:
loss = loss_func(outs, yb)

In [26]:
loss.backward()

In [27]:
bias.grad

tensor([-0.0160, -0.0304,  0.0461, -0.0306, -0.0361,  0.0333,  0.0237, -0.0068,
         0.0486, -0.0319])

In [33]:
lr = 0.5
epochs = 2
n, c = x_train.shape

for e in range(epochs):
     for ii in range((n - 1) // bs + 1):  # in batches of size bs, so roughly n / bs of them
        start_idx = ii * bs  # we are ii batches in, each of size bs
        end_idx = start_idx + bs  # and we want the next bs entires

        # pull batches from x and from y
        xb = x_train[start_idx:end_idx]
        yb = y_train[start_idx:end_idx]

        # run model
        pred = model(xb)

        # get loss
        loss = loss_func(pred, yb)
        print("happy:",loss)
        # calculate the gradients with a backwards pass
        loss.backward()

        with torch.no_grad():
            weights -= weights.grad*lr
            bias -= bias.grad*lr

            weights.grad.zero_()
            bias.grad.zero_()

happy: tensor(0.1473, grad_fn=<NegBackward0>)
happy: tensor(0.2041, grad_fn=<NegBackward0>)
happy: tensor(0.2827, grad_fn=<NegBackward0>)
happy: tensor(0.3148, grad_fn=<NegBackward0>)
happy: tensor(0.3352, grad_fn=<NegBackward0>)
happy: tensor(0.0528, grad_fn=<NegBackward0>)
happy: tensor(0.1539, grad_fn=<NegBackward0>)
happy: tensor(0.4180, grad_fn=<NegBackward0>)
happy: tensor(0.1372, grad_fn=<NegBackward0>)
happy: tensor(0.2704, grad_fn=<NegBackward0>)
happy: tensor(0.2866, grad_fn=<NegBackward0>)
happy: tensor(0.1680, grad_fn=<NegBackward0>)
happy: tensor(0.2577, grad_fn=<NegBackward0>)
happy: tensor(0.4882, grad_fn=<NegBackward0>)
happy: tensor(0.3887, grad_fn=<NegBackward0>)
happy: tensor(0.2547, grad_fn=<NegBackward0>)
happy: tensor(0.6062, grad_fn=<NegBackward0>)
happy: tensor(0.6744, grad_fn=<NegBackward0>)
happy: tensor(0.0946, grad_fn=<NegBackward0>)
happy: tensor(0.5623, grad_fn=<NegBackward0>)
happy: tensor(0.3665, grad_fn=<NegBackward0>)
happy: tensor(0.6110, grad_fn=<Neg

In [34]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))


tensor(0.0501, grad_fn=<NegBackward0>) tensor(1.)


In [36]:
# re-execute this cell for more samples
idx = random.randint(0, len(x_train))
example = x_train[idx:idx+1]

out = model(example)

print(out.argmax())
wandb.Image(example.reshape(28, 28)).image

tensor(5)
